In [1]:
import kernel

In [2]:
k = kernel.Kernel("linux-4.14.152/")
k.clean()
k.tinyconfig()
k.compile(time=True)
ctime = k.get_compile_time()
k.save("./backup")

In [3]:
ctime

36.6189054279821

In [4]:
ch = kernel.Checker(k, verbose=True)
ch.builtin()
vmsize = ch.vmlinux_size()
ch.dir_full_timestamp()

[c] CHECKER: BULT-IN...
[i] Checking for built-in.o
[x] CHECKER: BUILT-IN <DONE>
[c] CHECKER: VMLINUX SIZE
[x] CHECKER: VMLINUX SIZE <DONE>
[c] CHECKER: DIR TIMESTAMP
[x] CHECKER: DIR TIMESTAMP <DONE>


In [5]:
vmsize

1202432

In [6]:
import expdsl

In [8]:
sym, chains = expdsl.parse_file("test.cfg")

In [9]:
sym, chains

({'c': 'linux-4.14.152/.config'}, [['c']])

In [1]:
import itertools

In [7]:
perm = list(map(lambda x: " -> ".join(x), itertools.permutations(["config{}".format(i) for i in range(10)])))

In [200]:
test = []
i = 0
for ch in perm:
    if ch.split(' -> ')[0] == "config{}".format(i):
        test.append(perm[perm.index(ch) + 1])
        i += 1

In [12]:
perm[]

'config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config7 -> config9 -> config8 -> config6'

In [14]:
10!

SyntaxError: invalid syntax (<ipython-input-14-2d04f6099a43>, line 1)

In [16]:
test

['config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config6 -> config7 -> config8 -> config9',
 'config1 -> config0 -> config2 -> config3 -> config4 -> config5 -> config6 -> config7 -> config8 -> config9',
 'config2 -> config0 -> config1 -> config3 -> config4 -> config5 -> config6 -> config7 -> config8 -> config9',
 'config3 -> config0 -> config1 -> config2 -> config4 -> config5 -> config6 -> config7 -> config8 -> config9',
 'config4 -> config0 -> config1 -> config2 -> config3 -> config5 -> config6 -> config7 -> config8 -> config9',
 'config5 -> config0 -> config1 -> config2 -> config3 -> config4 -> config6 -> config7 -> config8 -> config9',
 'config6 -> config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config7 -> config8 -> config9',
 'config7 -> config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config6 -> config8 -> config9',
 'config8 -> config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config6 -> config7 -> config9',
 

In [201]:
for k in test:
    print(k)

config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config6 -> config7 -> config9 -> config8
config1 -> config0 -> config2 -> config3 -> config4 -> config5 -> config6 -> config7 -> config9 -> config8
config2 -> config0 -> config1 -> config3 -> config4 -> config5 -> config6 -> config7 -> config9 -> config8
config3 -> config0 -> config1 -> config2 -> config4 -> config5 -> config6 -> config7 -> config9 -> config8
config4 -> config0 -> config1 -> config2 -> config3 -> config5 -> config6 -> config7 -> config9 -> config8
config5 -> config0 -> config1 -> config2 -> config3 -> config4 -> config6 -> config7 -> config9 -> config8
config6 -> config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config7 -> config9 -> config8
config7 -> config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config6 -> config9 -> config8
config8 -> config0 -> config1 -> config2 -> config3 -> config4 -> config5 -> config6 -> config9 -> config7
config9 -> config0 -> config1 -> conf

In [18]:
for i in range(10):
    c = "config{}".format(i)
    print("{}: {}".format(c, c))

config0: config0
config1: config1
config2: config2
config3: config3
config4: config4
config5: config5
config6: config6
config7: config7
config8: config8
config9: config9


In [19]:
import kernel

In [20]:
k = kernel.Kernel("/tmp/linux-4.14.152/")

In [25]:
k.clean()

In [24]:
k.compile(dest="dest", config="config")

make -C /tmp/linux-4.14.152/ O=dest -j4


In [98]:
import pandas as pd
import os

In [197]:
def retrieve_tmstp(inc_dir, n):
    tmstp_files_list = ["{}/config{}/dir.tmstp".format(inc_dir, i) for i in range(n)]
    config_order_list = []
    with open("")
    return build_df(tmstp_files_list)

In [132]:
def parse_tmstp(tmstp_file):
    res = dict()
    with open(tmstp_file, 'r') as file:
        for line in file:
            l = line.split(" ")
            timestamp, name = l[0].strip(), "/".join(l[1].split("/")[2:]).strip("\n")
            if name == "":
                name = "LINUX_SRC_DIR"
            res[name] = timestamp
    del res["x64.config"]
    del res["dir.tmstp"]
    del res["builtins.size"]
    if "bloatto" in res:
        del res["bloatto"]
    return res

In [199]:
def build_df(list_files, config_order_list):
    data = [parse_tmstp(k) for k in list_files]
    state = data[0].copy()
    res = dict()
    for i, t_file in enumerate(data):
        for f in t_file:
            if f in res:
                if state[f] == t_file[f]:
                    res[f].append(0)
                else:
                    state[f] = t_file[f]
                    res[f].append(1)
            else:
                state[f] = t_file[f]
                res[f] = [0 for j in range(i)]
                res[f].append(1)
#         if some files are not in t_files, it means that they are not
#         in the source directory anymore: they were removed. -1 means
#         it was removed
        for f in res:
            lon = len(res[f])
            ri = i + 1
            if lon < ri:
                res[f].extend([-1 for j in range(ri-lon)])
    return pd.DataFrame.from_dict(res, index=config_order_list)

In [190]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 156680 entries, LINUX_SRC_DIR to sum
dtypes: int64(156680)
memory usage: 12.0 MB


In [203]:
"OOP"[::-1]

'POO'